In [1]:
import requests
import json
from pandas import json_normalize
import pandas as pd
from owlready2 import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
df = pd.read_csv('case.csv')
df.head()

,id,patient_id,gene_id,features
0,patient_1,Patient:SCV000492960,Entrez:10000,"HP:0005306,HP:0000256,HP:0001263,HP:0002126"
1,patient_2,Patient:SCV000747622,Entrez:10002,"HP:0000505,HP:0000666,HP:0000551,HP:0000556"
2,patient_3,Patient:SCV000599197,Entrez:10002,HP:0000556
3,patient_4,Patient:SCV000598737,Entrez:1001,HP:0007754
4,patient_5,Patient:293197,Entrez:100151683,"HP:0001631,HP:0001601,HP:0011344,HP:0011994"


In [15]:
# trial
q = df['features'][0]
query = q.replace(",", ";")
para = {'q': query}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=para).json()
concept_id = json_normalize(response['results'])
disease = concept_id['concept_name'].values.tolist()

In [23]:
disease = []
for i in range(0, 4714):
    feat_list = df['features'][i]
    query = feat_list.replace(",", ";")
    param = {'q': query}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params=param).json()
    concept_id = json_normalize(response['results'])
    result = concept_id['concept_name'].values.tolist()
    disease.append(result)

In [43]:
df['disease'] = disease
df

,id,patient_id,gene_id,features,disease
0,patient_1,Patient:SCV000492960,Entrez:10000,"HP:0005306,HP:0000256,HP:0001263,HP:0002126","[Polymicrogyria, Global developmental delay, M..."
1,patient_2,Patient:SCV000747622,Entrez:10002,"HP:0000505,HP:0000666,HP:0000551,HP:0000556","[inherited retinal dystrophy, Horizontal nysta..."
2,patient_3,Patient:SCV000599197,Entrez:10002,HP:0000556,"[inherited retinal dystrophy, Retinal dystrophy]"
3,patient_4,Patient:SCV000598737,Entrez:1001,HP:0007754,[Macular dystrophy]
4,patient_5,Patient:293197,Entrez:100151683,"HP:0001631,HP:0001601,HP:0011344,HP:0011994","[Severe global developmental delay, Laryngomal..."
...,...,...,...,...,...
4709,patient_4710,Patient:235427,Entrez:999,"HP:0000656,HP:0100336,HP:0009743,HP:0000698,HP...","[Delayed eruption of teeth, Lagopthalmos, Hype..."
4710,patient_4711,Patient:235667,Entrez:999,"HP:0100333,HP:0000692,HP:0009755,HP:0025248","[Ankyloblepharon, Tooth malposition, Eruptive ..."
4711,patient_4712,Patient:tubingen236,Entrez:9990,HP:0001271,[Polyneuropathy]
4712,patient_4713,Patient:SCV000747695,Entrez:9990,"HP:0001274,HP:0000369,HP:0004209,HP:0000316,HP...","[Hypertelorism, Low-set ears, Clinodactyly of ..."


In [45]:
df.to_csv('phen_disease.csv')